# 1️⃣ Clone MeshMind Repository
This cell clones the MeshMind repository into Colab and navigates into it.  
All subsequent setup steps will operate from this directory.

In [ ]:
!git clone https://github.com/dimp01/MeshMind /content/MeshMind/
%cd /content/MeshMind/

# 2️⃣ Install System Dependencies
PyVista and VTK require OpenGL libraries and a virtual framebuffer for headless rendering.
This cell installs the necessary Linux packages.


In [ ]:
!apt-get update
!apt-get install -y libgl1 libglib2.0-0 xvfb mesa-utils
!apt-get install -y libosmesa6-dev
!apt-get install -y xvfb

# 3️⃣ Install Python Dependencies
Install all required Python packages from `requirements.txt`.  
This includes Streamlit, PyTorch, diffusers, PyVista, stpyvista, and more.


In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/facebookresearch/pytorch3d.git@stable -vv
!pip install -r requirements.txt

# 4️⃣ Run Setup
If `.env` file does not exist, this cell runs `setup.py` to configure:
- GEMINI_API_KEY
- NGROK_AUTH_TOKEN

This ensures the app has the required credentials.

In [ ]:
import os
if not os.path.exists(".env"):
    print("⚡ .env file not found. Running setup.py to create it.")
    !python setup.py
else:
    print("✅ .env file found. Skipping setup.py.")

# 5️⃣ Load Environment Variables
Load `.env` file so the GEMINI_API_KEY and NGROK_AUTH_TOKEN are available in the environment.

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Automatically sets os.environ

# 6️⃣ Configure PyVista for Headless Rendering
PyVista requires off-screen rendering in Colab/Docker.  
This prevents OpenGL errors when running Streamlit.

In [ ]:
import pyvista as pv
pv.start_xvfb()  # off-screen rendering

# 7️⃣ Start Streamlit and ngrok
This cell:
1. Launches Streamlit in the background.
2. Opens a public ngrok tunnel.
3. Prints the public URL for accessing the app.

In [ ]:
from pyngrok import ngrok
import time
import os

# Set your ngrok authtoken
ngrok.set_auth_token(os.environ.get("NGROK_AUTH_TOKEN"))
ngrok.kill()

# Train Model
%run training/train.py

port = 8501
os.environ["STREAMLIT_SERVER_HEADLESS"] = "true"

# Open ngrok tunnel
public_url = ngrok.connect(port)
print("🌐 Streamlit is running at:", public_url)

# Launch Streamlit
!streamlit run main.py --server.port {port} --server.headless true
ngrok.kill()             # Close ngrok tunnels


# 8️⃣ Pull Updates from Git Repo
Use this cell to pull latest updates from git repo.
Rerun all the cells of the notebook after updating.

In [ ]:
!git pull             # pull latest updates